## Books Recommender System using CLustering | Collaborative Based Filtering

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [94]:
books = pd.read_csv('data/Books.csv', encoding='latin-1')

C:\Users\Mukand\AppData\Local\Temp\ipykernel_13140\700269677.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='latin-1')


In [95]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [96]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [97]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [98]:
books.rename(columns={"Book-Title":"Title",
                     "Book-Author": 'Author',
                     'Year-Of-Publication':'Year',
                     'Image-URL-L':'Img-URL'},
            inplace=True)

C:\Users\Mukand\AppData\Local\Temp\ipykernel_13140\66292342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.rename(columns={"Book-Title":"Title",


In [100]:
books.shape

(271360, 6)

In [101]:
users = pd.read_csv('data/Users.csv', encoding='latin-1')

In [102]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [103]:
users.rename(columns={'User-ID':'ID'}, inplace=True)

In [104]:
ratings = pd.read_csv('data/Ratings.csv', encoding='latin-1')

In [105]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,155061224.00,5
2,276727,446520802.00,0
3,276729,052165615X,3
4,276729,521795028.00,6


In [106]:
ratings.rename(columns={'User-ID':'ID', 'Book-Rating':'Rating'}, inplace=True)

shapes

In [107]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 6)
(1048575, 3)
(278858, 3)


In [108]:
ratings['ID'].value_counts()

ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
104999        1
105002        1
105008        1
105014        1
123969        1
Name: count, Length: 95513, dtype: int64

In [109]:
ratings['ID'].unique().shape

(95513,)

#### those users who has rated more than 200 books

In [110]:
x=ratings['ID'].value_counts()>200
x[x].shape

(815,)

In [111]:
y=x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
        88793,  33145, 116122,   9856,  73681,  28634,  59727, 188951, 155916,
        44296],
      dtype='int64', name='ID', length=815)

In [112]:
ratings = ratings[ratings['ID'].isin(y)]

In [114]:
ratings.shape

(482728, 3)

In [115]:
ratings_with_books = ratings.merge(books, on="ISBN")

In [117]:
ratings_with_books.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [119]:
ratings_with_books.shape

(37455, 8)

In [121]:
num_rating = ratings_with_books.groupby('Title')['Rating'].count().reset_index()

In [123]:
num_rating.head()

,Title,Rating
0,Earth Prayers From around the World: 365 Pray...,3
1,Flight of Fancy: American Heiresses (Zebra Ba...,1
2,Little Comic Shop of Horrors (Give Yourself G...,2
3,"Q-Zone (Star Trek The Next Generation, Book 48)",7
4,The Town Cats and Other Tales,1


In [125]:
num_rating.rename(columns={'Rating':'Num_of_rating'}, inplace=True)

In [127]:
num_rating

,Title,Num_of_rating
0,Earth Prayers From around the World: 365 Pray...,3
1,Flight of Fancy: American Heiresses (Zebra Ba...,1
2,Little Comic Shop of Horrors (Give Yourself G...,2
3,"Q-Zone (Star Trek The Next Generation, Book 48)",7
4,The Town Cats and Other Tales,1
...,...,...
13660,the Dark Light Years,1
13661,together by christmas,6
13662,voyage fatal,1
13663,why I'm like this : True Stories,4


In [128]:
final_rating = ratings_with_books.merge(num_rating, on ='Title')

In [129]:
final_rating.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL,Num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74


In [130]:
final_rating.shape

(37455, 9)

#### book which got more than 50 ratings

In [131]:
final_rating = final_rating[final_rating['Num_of_rating']>=50]

In [134]:
final_rating.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL,Num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74


In [135]:
final_rating.shape

(4297, 9)

In [137]:
# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='ID', index='Title', values= 'Rating')

In [141]:
book_pivot

ID,254,2276,2766,2977,3363,4017,6242,6251,6323,6543,...,249628,249862,249894,250184,250405,250764,277427,277478,277639,278418
Title,,,,,,,,,,,,,,,,,,,,,
A Fine Balance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Painted House,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Patchwork Planet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All I Really Need to Know,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Angela's Ashes: A Memoir,0.0,0.0,0.0,0.0,0.0,10.0,8.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anne of Green Gables (Anne of Green Gables Novels (Paperback)),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
At Home in Mitford (The Mitford Years),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Atonement : A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bag of Bones,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
book_pivot.fillna(0, inplace=True)

### Training Model

In [143]:
from scipy.sparse import csr_matrix

In [145]:
book_sparse = csr_matrix(book_pivot)

In [146]:
book_sparse

<53x724 sparse matrix of type '<class 'numpy.float64'>'
	with 1057 stored elements in Compressed Sparse Row format>

In [147]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [148]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [154]:
distance, suggestion = model.kneighbors(book_pivot.iloc[43,:].values.reshape(1,-1), n_neighbors=6 )

In [155]:
distance

array([[ 0.        , 33.76388603, 35.98958183, 37.60319135, 37.68288736,
        38.09199391]])

In [156]:
suggestion

array([[43, 46,  2, 32, 27, 14]], dtype=int64)

In [163]:
book_pivot.iloc[51,:]

ID
254       0.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
250764    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: What Looks Like Crazy On An Ordinary Day, Length: 724, dtype: float64

In [164]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['The Lost World', 'The Reef', 'A Patchwork Planet', 'Red Storm Rising',
       'Men Are from Mars, Women Are from Venus: A Practical Guide for Improving Communication and Getting What You Want in Your Relationships',
       'Degree of Guilt'],
      dtype='object', name='Title')


In [166]:
book_pivot.index[5]

'Anne of Green Gables (Anne of Green Gables Novels (Paperback))'

In [167]:
book_names = book_pivot.index

In [168]:
book_names[2]

'A Patchwork Planet'

In [170]:
np.where(book_pivot.index == 'Anne of Green Gables (Anne of Green Gables Novels (Paperback))'
)[0][0]

5

### finding url

In [173]:
ids = np.where(final_rating['Title'] == "The Lost World")[0][0]


In [177]:
final_rating.iloc[ids]['Img-URL']

'http://images.amazon.com/images/P/034540288X.01.LZZZZZZZ.jpg'

In [178]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])

In [179]:
book_name[0]

Index(['The Lost World', 'The Reef', 'A Patchwork Planet', 'Red Storm Rising',
       'Men Are from Mars, Women Are from Venus: A Practical Guide for Improving Communication and Getting What You Want in Your Relationships',
       'Degree of Guilt'],
      dtype='object', name='Title')

In [182]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['Title'] == name)[0][0]
    ids_index.append(ids)

In [183]:
for idx in ids_index:
    url = final_rating.iloc[idx]['Img-URL']
    print(url)

http://images.amazon.com/images/P/034540288X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/051512608X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/080411918X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/042510107X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/006016848X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/034538184X.01.LZZZZZZZ.jpg


In [194]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

### Testing model

In [185]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [193]:
book_name = "A Fine Balance"
recommend_book(book_name)

You searched 'A Fine Balance'

The suggestion books are: 

The Reef
A Patchwork Planet
Degree of Guilt
The Girls' Guide to Hunting and Fishing
Red Storm Rising
